미니배치, 이터레이션, 에포크 참조  
https://wikidocs.net/55580

In [166]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torch.utils.data import DataLoader # 데이터로더
from torch.utils.data import Dataset

In [167]:
class TensorDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

      # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)

      # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [168]:
xy = np.loadtxt('./data/diabetes.csv.gz', delimiter=',', dtype=np.float32)

In [169]:
class DiabetesDataset(Dataset):
    
    def __init__(self, xy_dataset):
        # 커스텀 데이터셋 클래스의 생성자
        # 데이터를 불러오고 전처리 한다.
        self.x_data = torch.from_numpy(xy_dataset[:, 0:-1])
        self.y_data = torch.from_numpy(xy_dataset[:, [-1]])
        print(f'X\'s shape: {self.x_data.shape} | Y\'s shape: {self.y_data.shape}')

    def __len__(self):
        # 데이터셋의 길이를 반환하는 메소드
        return len(self.x_data)
        
    def __getitem__(self, idx):
        # 데이터셋에서 하나의 특정 샘플 (x, y)을 가져오는 메소드
        return self.x_data[idx], self.y_data[idx]

In [170]:
# 구현한 커스텀 데이터셋 클래스의 인스턴스 생성
dataset = DiabetesDataset(xy)

X's shape: torch.Size([759, 8]) | Y's shape: torch.Size([759, 1])


In [171]:
# DataLoader 클래스를 사용하여 데이터를 불러온 후, 모델에 입력하게 된다.
data_loader = DataLoader(dataset, batch_size=100)

## 퀴즈 (Easy)  
1) 당뇨병 데이터셋에 대해서 로지스틱 회귀 모델을 구현한다면 첫번째 레이어의 노드 수는 몇개가 되어야 할까요??   
2) data_loader에서 샘플들을 가져온다면 샘플들의 전체 모양은 어떻게 될까요?  

## 퀴즈 (Normal)  
이전의 노트북 파일을 참고해서 LogisticRegressionModel 클래스를 구현하세요.  
1) 생성자 :  
모델의 은닉층 수는 3층이고 은닉층마다 (8, 6, 4) 개의 뉴런을 가집니다.  
마지막 레이어에는 시그모이드 모듈을 이용해 예측값을 계산하도록 만드세요.  
2) forward :   
최종적으로 확률 값을 예측하도록 레이어를 쌓아서 y를 반환하세요.  

In [163]:
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.L1 = nn.Linear(8, 6)
        self.L2 = nn.Linear(6, 4)
        self.L3 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        z1 = self.sigmoid(self.L1(x))
        z2 = self.sigmoid(self.L2(z1))
        z3 = self.sigmoid(self.L3(z2))
        y_pred = self.sigmoid(z3)
        return y_pred



## 퀴즈 (Normal)  
1) 위 모델을 학습시키기 위해서는 어떤 손실함수를 선택해야할까요??  
2) 위의 로지스틱회귀 클래스 모델을 조금 더 메모리 효율적으로 구현하려면 어떻게 해야할까요?  
3) 위의 로지스틱 회귀 모델을 공책에 그려봅시다. 

In [164]:
model = LogisticRegressionModel()
bce_loss = nn.BCELoss(reduction='mean')
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)


In [165]:
epochs = 1000

for epoch in range(epochs):
    for batch_idx, samples in enumerate(data_loader):
        x_train, y_train = samples
        y_pred = model(x_train)
        loss = bce_loss(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch: {epoch+1:4d}/{epochs} Batch {batch_idx+1:4d}/{len(data_loader)} \
                Loss: {loss.item():4f}')


Epoch:    1/1000 Batch    1/8                     Loss: 0.641854
Epoch:    2/1000 Batch    1/8                     Loss: 0.639658
Epoch:    3/1000 Batch    1/8                     Loss: 0.636665
Epoch:    4/1000 Batch    1/8                     Loss: 0.628623
Epoch:    5/1000 Batch    1/8                     Loss: 0.610536
Epoch:    6/1000 Batch    1/8                     Loss: 0.584162
Epoch:    7/1000 Batch    1/8                     Loss: 0.564793
Epoch:    8/1000 Batch    1/8                     Loss: 0.556183
Epoch:    9/1000 Batch    1/8                     Loss: 0.553297
Epoch:   10/1000 Batch    1/8                     Loss: 0.552558
Epoch:   11/1000 Batch    1/8                     Loss: 0.553340
Epoch:   12/1000 Batch    1/8                     Loss: 0.555216
Epoch:   13/1000 Batch    1/8                     Loss: 0.557112
Epoch:   14/1000 Batch    1/8                     Loss: 0.558286
Epoch:   15/1000 Batch    1/8                     Loss: 0.558863
Epoch:   16/1000 Batch   

Epoch:  140/1000 Batch    1/8                     Loss: 0.551827
Epoch:  141/1000 Batch    1/8                     Loss: 0.551789
Epoch:  142/1000 Batch    1/8                     Loss: 0.551751
Epoch:  143/1000 Batch    1/8                     Loss: 0.551713
Epoch:  144/1000 Batch    1/8                     Loss: 0.551676
Epoch:  145/1000 Batch    1/8                     Loss: 0.551638
Epoch:  146/1000 Batch    1/8                     Loss: 0.551601
Epoch:  147/1000 Batch    1/8                     Loss: 0.551563
Epoch:  148/1000 Batch    1/8                     Loss: 0.551526
Epoch:  149/1000 Batch    1/8                     Loss: 0.551489
Epoch:  150/1000 Batch    1/8                     Loss: 0.551452
Epoch:  151/1000 Batch    1/8                     Loss: 0.551415
Epoch:  152/1000 Batch    1/8                     Loss: 0.551377
Epoch:  153/1000 Batch    1/8                     Loss: 0.551340
Epoch:  154/1000 Batch    1/8                     Loss: 0.551304
Epoch:  155/1000 Batch   

Epoch:  270/1000 Batch    1/8                     Loss: 0.547498
Epoch:  271/1000 Batch    1/8                     Loss: 0.547465
Epoch:  272/1000 Batch    1/8                     Loss: 0.547431
Epoch:  273/1000 Batch    1/8                     Loss: 0.547396
Epoch:  274/1000 Batch    1/8                     Loss: 0.547362
Epoch:  275/1000 Batch    1/8                     Loss: 0.547327
Epoch:  276/1000 Batch    1/8                     Loss: 0.547292
Epoch:  277/1000 Batch    1/8                     Loss: 0.547257
Epoch:  278/1000 Batch    1/8                     Loss: 0.547221
Epoch:  279/1000 Batch    1/8                     Loss: 0.547185
Epoch:  280/1000 Batch    1/8                     Loss: 0.547149
Epoch:  281/1000 Batch    1/8                     Loss: 0.547113
Epoch:  282/1000 Batch    1/8                     Loss: 0.547076
Epoch:  283/1000 Batch    1/8                     Loss: 0.547039
Epoch:  284/1000 Batch    1/8                     Loss: 0.547001
Epoch:  285/1000 Batch   

Epoch:  411/1000 Batch    1/8                     Loss: 0.545376
Epoch:  412/1000 Batch    1/8                     Loss: 0.544028
Epoch:  413/1000 Batch    1/8                     Loss: 0.544112
Epoch:  414/1000 Batch    1/8                     Loss: 0.544641
Epoch:  415/1000 Batch    1/8                     Loss: 0.544343
Epoch:  416/1000 Batch    1/8                     Loss: 0.544218
Epoch:  417/1000 Batch    1/8                     Loss: 0.544336
Epoch:  418/1000 Batch    1/8                     Loss: 0.544505
Epoch:  419/1000 Batch    1/8                     Loss: 0.545756
Epoch:  420/1000 Batch    1/8                     Loss: 0.543956
Epoch:  421/1000 Batch    1/8                     Loss: 0.544309
Epoch:  422/1000 Batch    1/8                     Loss: 0.544440
Epoch:  423/1000 Batch    1/8                     Loss: 0.546065
Epoch:  424/1000 Batch    1/8                     Loss: 0.545409
Epoch:  425/1000 Batch    1/8                     Loss: 0.545296
Epoch:  426/1000 Batch   

Epoch:  552/1000 Batch    1/8                     Loss: 0.543940
Epoch:  553/1000 Batch    1/8                     Loss: 0.543951
Epoch:  554/1000 Batch    1/8                     Loss: 0.543944
Epoch:  555/1000 Batch    1/8                     Loss: 0.543954
Epoch:  556/1000 Batch    1/8                     Loss: 0.543951
Epoch:  557/1000 Batch    1/8                     Loss: 0.543943
Epoch:  558/1000 Batch    1/8                     Loss: 0.543948
Epoch:  559/1000 Batch    1/8                     Loss: 0.543949
Epoch:  560/1000 Batch    1/8                     Loss: 0.543945
Epoch:  561/1000 Batch    1/8                     Loss: 0.543944
Epoch:  562/1000 Batch    1/8                     Loss: 0.543944
Epoch:  563/1000 Batch    1/8                     Loss: 0.543942
Epoch:  564/1000 Batch    1/8                     Loss: 0.543940
Epoch:  565/1000 Batch    1/8                     Loss: 0.543938
Epoch:  566/1000 Batch    1/8                     Loss: 0.543936
Epoch:  567/1000 Batch   

Epoch:  691/1000 Batch    1/8                     Loss: 0.546993
Epoch:  692/1000 Batch    1/8                     Loss: 0.547009
Epoch:  693/1000 Batch    1/8                     Loss: 0.547135
Epoch:  694/1000 Batch    1/8                     Loss: 0.548386
Epoch:  695/1000 Batch    1/8                     Loss: 0.551514
Epoch:  696/1000 Batch    1/8                     Loss: 0.547570
Epoch:  697/1000 Batch    1/8                     Loss: 0.547649
Epoch:  698/1000 Batch    1/8                     Loss: 0.544760
Epoch:  699/1000 Batch    1/8                     Loss: 0.544524
Epoch:  700/1000 Batch    1/8                     Loss: 0.545776
Epoch:  701/1000 Batch    1/8                     Loss: 0.543593
Epoch:  702/1000 Batch    1/8                     Loss: 0.543755
Epoch:  703/1000 Batch    1/8                     Loss: 0.543960
Epoch:  704/1000 Batch    1/8                     Loss: 0.544416
Epoch:  705/1000 Batch    1/8                     Loss: 0.543736
Epoch:  706/1000 Batch   

Epoch:  832/1000 Batch    1/8                     Loss: 0.543245
Epoch:  833/1000 Batch    1/8                     Loss: 0.543244
Epoch:  834/1000 Batch    1/8                     Loss: 0.543244
Epoch:  835/1000 Batch    1/8                     Loss: 0.543244
Epoch:  836/1000 Batch    1/8                     Loss: 0.543243
Epoch:  837/1000 Batch    1/8                     Loss: 0.543243
Epoch:  838/1000 Batch    1/8                     Loss: 0.543243
Epoch:  839/1000 Batch    1/8                     Loss: 0.543242
Epoch:  840/1000 Batch    1/8                     Loss: 0.543242
Epoch:  841/1000 Batch    1/8                     Loss: 0.543242
Epoch:  842/1000 Batch    1/8                     Loss: 0.543242
Epoch:  843/1000 Batch    1/8                     Loss: 0.543242
Epoch:  844/1000 Batch    1/8                     Loss: 0.543242
Epoch:  845/1000 Batch    1/8                     Loss: 0.543241
Epoch:  846/1000 Batch    1/8                     Loss: 0.543241
Epoch:  847/1000 Batch   

Epoch:  971/1000 Batch    1/8                     Loss: 0.545056
Epoch:  972/1000 Batch    1/8                     Loss: 0.554024
Epoch:  973/1000 Batch    1/8                     Loss: 0.555855
Epoch:  974/1000 Batch    1/8                     Loss: 0.548046
Epoch:  975/1000 Batch    1/8                     Loss: 0.548077
Epoch:  976/1000 Batch    1/8                     Loss: 0.553565
Epoch:  977/1000 Batch    1/8                     Loss: 0.558369
Epoch:  978/1000 Batch    1/8                     Loss: 0.549479
Epoch:  979/1000 Batch    1/8                     Loss: 0.548892
Epoch:  980/1000 Batch    1/8                     Loss: 0.564105
Epoch:  981/1000 Batch    1/8                     Loss: 0.555451
Epoch:  982/1000 Batch    1/8                     Loss: 0.547035
Epoch:  983/1000 Batch    1/8                     Loss: 0.547451
Epoch:  984/1000 Batch    1/8                     Loss: 0.548275
Epoch:  985/1000 Batch    1/8                     Loss: 0.547035
Epoch:  986/1000 Batch   

## 퀴즈 (Easy)  
위 학습 반복문에서 enumerate를 사용했습니다. 그 이유가 무엇일까요?  